<h1 style="text-align:center; font-size:200%;">Lab 2</h1>
<h1 style="text-align:center; font-size:150%;">Bill Generation</h1>
<h4 style="text-align:left;">Performed by: DAGHMOUMI Marouan</h4>
<h4 style="text-align:left;">Supervised by: AACHAK Lotfi</h4>


# *Part 1: Rule Based NLP and Regex:*



### 1.**Importation des biblio**



In [1]:
from word2number import w2n
from tabulate import tabulate
import re
from nltk.corpus import stopwords
import spacy



### 2.**Declaration**



In [2]:
numbers = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety', 'hundred', 'thousand', 'million']
pattern = r"((?:" + '|'.join(numbers) + r"|\d)(?:\s(?:" + '|'.join(numbers) + r"|\d|and))*)(.*?)(\d+[\.|\,]?\d*)\b\s*(\$|dollar)"

In [3]:
text = "I bought three Samsung smartphones 150 $ each, four kilos of fresh banana for 1,2 dollar a kilogram and one Hamburger with 4,5 dollar "



### 3.**NLP pipline**



In [5]:
# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Tokenize the text
word_tokens = nlp(text)

# Filter out tokens that are not stop words or adjectives
filtered_tokens = [token.text for token in word_tokens if token.pos_ != "ADJ"]

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Filter out stop words
filtered_tokens = [token for token in filtered_tokens if token.lower() not in stop_words]
filtered_tokens = ' '.join(filtered_tokens)

# Define regex pattern to match weight units and words ending with "gram" or "grams"
weight_units_regex = r'\b(?:kg|kilos|lb|pound|[\w]*(?:gram|grams))\b'
filtered_tokens = re.sub(weight_units_regex, '', filtered_tokens)

### 4.**Rule based**


In [6]:
matches = re.finditer(pattern, filtered_tokens)

# Initialize lists to store data for each column
products = []
quantities = []
unit_prices = []
total_prices = []

# Process each match
for match in matches:
    quantity = match.group(1)
    # Remove commas from the quantity string and convert it to a number
    quantity = w2n.word_to_num(quantity.replace(',', '.'))
    product = match.group(2)
    price = float(match.group(3).replace(',', '.'))  # Remove commas from the price string
    
    # Calculate total price
    total_price = quantity * price
    
    products.append(product)
    quantities.append(quantity)
    unit_prices.append(price)
    total_prices.append(total_price)

bill_data = list(zip(products, quantities, unit_prices, total_prices))

### 5.**Resultat**


In [7]:
bill_data

[(' Samsung smartphones ', 3, 150.0, 450.0),
 ('  banana ', 4, 1.2, 4.8),
 (' Hamburger ', 1, 4.5, 4.5)]

# *Part 2: word Embedding*

* import data

In [14]:
import pymongo
# Connect to MongoDB
client = pymongo.MongoClient('localhost', 27017)
db = client['Atelier1']
collection = db['News_data']

# Fetch content from MongoDB
cursor = collection.find()
content_list = []

In [15]:
count = 0
# Assuming you have already defined the 'cursor' object
for document in cursor:
    if count >= 1:
        break  # Exit the loop if we've processed two contents
    content = document.get('content')
    content_list.append(content)
    count += 1

In [18]:
print(content)

بعد يوم واحد من مطالبة منظمة العفو الدولية، السلطات الإسرائيلية بالإفراج عن السجين الفلسطيني وليد دقة البالغ من العمر 62 عاماً، أعلن نادي الأسير الفلسطيني عن وفاة وليد داخل السجن متأثراً بإصابته بمرض السرطان. وقالت منظمة العفو الدولية إن وليد دقة مصاب بـ"مرض عضال"، إذ شُخِّصت إصابته بسرطان النخاع الشوكي، والذي يُعَد من الحالات الطبية النادرة. ومنذ السابع من أكتوبر/تشرين الأول 2023، تعرَّض وليد دقة للتعذيب والإهانة وحُرِم من زيارات عائلته، إضافةً إلى تعرُّضه للإهمال الطبي، ونقل خلال الفترة الماضية إلى المستشفى مرتين بسبب تدهور حالته الصحية، بحسب منظمة العفو الدولية. ويقضي دقة حكماً بالسجن مدة 38 عاماً، بعد اتهامه بالمشاركة "مع جماعة مسلحة اختطفت وقتلت جندياً إسرائيلياً عام 1984" وهو موشيه تمام. وتقول منظمة العفو الدولية في تقرير صدر عنها العام الماضي إنه كان من المفترض انتهاء محكومية دقة في مارس/آذار 2023، لكن حكم عليه في محكمة عسكرية عام 2018 بالسجن عامين إضافيين بتهمة محاولة تهريب هواتف نقالة إلى سجناء فلسطينيين آخرين. وأعلنت هيئة شؤون الأسرى الفلسطينية وفاة السجين الفلسطيني وليد دقة 

* One-Hot Encoding

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
# Tokenize the text content into words
tokenized_content = [content.split() for content in content_list]

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the tokenized data
one_hot_encoded = encoder.fit_transform(tokenized_content)

# Convert the sparse matrix to a DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.categories_[0])

print("One-hot Encoded DataFrame:")
print(one_hot_encoded_df)  

ValueError: Shape of passed values is (1, 473), indices imply (1, 1)